<a href="https://colab.research.google.com/github/kabeerbora/cs_ls/blob/main/state_maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Necessary Shape Files

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
!pip install geopandas==0.12.2
import geopandas as gpd
import matplotlib.pyplot as plt

# Download the shapefile and associated files directly
!wget -q -O India_State_Boundary.shp https://github.com/AnujTiwari/India-State-and-Country-Shapefile-Updated-Jan-2020/raw/master/India_State_Boundary.shp
!wget -q -O India_State_Boundary.shx https://github.com/AnujTiwari/India-State-and-Country-Shapefile-Updated-Jan-2020/raw/master/India_State_Boundary.shx
!wget -q -O India_State_Boundary.dbf https://github.com/AnujTiwari/India-State-and-Country-Shapefile-Updated-Jan-2020/raw/master/India_State_Boundary.dbf

# Load the shapefile
india = gpd.read_file("India_State_Boundary.shp")

AttributeError: module 'fiona' has no attribute 'path'

## Create variables

In [ ]:
df_total = pd.read_excel('/content/drive/MyDrive/Bora-Sarangi/allstates_allvariables.xlsx', sheet_name = 'states_added')
df_total['year'] = df_total['year'].astype(str).str.split('-').str[0].astype(int)
df_total['labor_share'] = df_total['Wages and Salaries - Workers'] / df_total['Net Value Added']
df_total['contract_share'] = df_total['Number of Workers - Employed Through Contractors'] / df_total['Number of Workers']
df_total['divergence'] = df_total['Wages and Salaries - Workers']/df_total['Net Value Added']
df_total

In [ ]:
for state, group in df_total.groupby('state'):
    # Filter out rows with 0 or NA in labor_share, contract_share, and divergence
    filtered_group = group[group['labor_share'].notna() & (group['labor_share'] != 0) &
                           group['contract_share'].notna() & (group['contract_share'] != 0) &
                          group['divergence'].notna() & (group['divergence'] != 0)]  # Added divergence filter

    # Calculate percentage change on the filtered group
    filtered_group['delta_ls_pct'] = filtered_group['labor_share'].pct_change() * 100
    filtered_group['delta_cs_pct'] = filtered_group['contract_share'].pct_change() * 100
    filtered_group['delta_div_pct'] = filtered_group['divergence'].pct_change() * 100  # Calculate divergence percentage change
    # df_total.loc[filtered_group.index, ['delta_ls_pct', 'delta_cs_pct']] = filtered_group[['delta_ls_pct', 'delta_cs_pct']]
    # Assign the calculated values back to the original DataFrame
    df_total.loc[filtered_group.index, ['delta_ls_pct', 'delta_cs_pct', 'delta_div_pct']] = filtered_group[['delta_ls_pct', 'delta_cs_pct', 'delta_div_pct']] # Include delta_div_pct


<ipython-input-9-ea038bf29e90>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group['delta_ls_pct'] = filtered_group['labor_share'].pct_change() * 100
<ipython-input-9-ea038bf29e90>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group['delta_cs_pct'] = filtered_group['contract_share'].pct_change() * 100
<ipython-input-9-ea038bf29e90>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [ ]:
avg_by_ind_code = df_total.groupby('state').agg(lambda x: x[x.notna() & (x != 0)].mean() if x.dtype == float or x.dtype == int else None)
avg_by_ind_code.sample()

,year,CPI,Additions to Total Stock,Depreciation,Fixed Capital,Gross Fixed Capital Formation,Gross Value Added,Interest Paid,Invested Capital,Net Fixed Capital Formation,...,Wages and Salaries - Supervisory and Managerial Staff,Wages and Salaries - Total,Wages and Salaries - Workers,Working Capital,labor_share,contract_share,divergence,delta_ls_pct,delta_cs_pct,delta_div_pct
state,,,,,,,,,,,,,,,,,,,,,
BR,2000.756098,743.716167,206275.756098,500480.097561,7.740623e+06,917352.756098,3.167308e+06,414535.02439,1.018130e+07,416873.560976,...,331516.375,1.048452e+06,413901.341463,1.281681e+06,0.20831,0.401325,0.20831,4.539081,5.351141,4.539081


## Mapping states to our shape file

In [ ]:
state_mapping = {
    'Telengana': 'AP',
    'Andhra Pradesh': 'AP',
    'Chhattishgarh': 'MP',
    'Madhya Pradesh': 'MP',
    'Jharkhand': 'BR',
    'Bihar': 'BR',
    'Uttarakhand': 'UP',
    'Uttar Pradesh': 'UP',
    'Tamilnadu': 'Tamil Nadu',
     'Odisha': 'Orissa',
}
india['State_Name'] = india['State_Name'].replace(state_mapping)
india = india.dissolve(by='State_Name', aggfunc='first')
india = india.reset_index()

NameError: name 'india' is not defined

In [ ]:
import matplotlib.pyplot as plt
merged_data = india.merge(avg_by_ind_code, left_on='State_Name', right_on='state', how='left')


variable_to_map = 'labor_share'  # Or any other variable you want to map #Removed the extra spaces in the beginning of this line

fig, ax = plt.subplots(1, figsize=(12, 12))
merged_data.plot(column=variable_to_map,
                 cmap='viridis',  # Choose a colormap
                 linewidth=0.8,
                 ax=ax,
                 edgecolor='0.8',
                 legend=True,
                 missing_kwds={'color': 'lightgrey'})  # Customize missing values
ax.set_title('Average Labor Share by State')
ax.axis('off')
plt.show()